In [1]:
# librerías
import pprint
import pymongo
import numpy as np
import pandas as pd
from bson.son import SON
from pymongo import  GEO2D
import plotly.express as px
import plotly.graph_objects as go

# Bases de datos limpias

In [50]:
df_estacion=pd.read_csv('./Datos/datos_limpios/datos_estacion.csv').drop(columns='Unnamed: 0').rename(columns={'start station id':'id_estacion','start station name':'nombre_estacion','start station latitude':'lat','start station longitude':'long'})
df_estacion.head()

,id_estacion,nombre_estacion,lat,long
0,294,Washington Square E,40.730494,-73.995721
1,285,Broadway & E 14 St,40.734546,-73.990741
2,247,Perry St & Bleecker St,40.735354,-74.004831
3,357,E 11 St & Broadway,40.732618,-73.991580
4,401,Allen St & Rivington St,40.720196,-73.989978


In [51]:
df_viajes = pd.read_csv('./Datos/datos_limpios/datos_viajes.csv', index_col=0)
df_viajes = df_viajes.rename(columns={'start station id':'start_id',
                                      'end station id':'end_id',
                                      'tripduration':'duracion_prom'})
df_viajes = df_viajes.drop(df_viajes[df_viajes['start_id'] == df_viajes['end_id']].index)
df_viajes.head()

,start_id,end_id,duracion_prom
0,72,72,1428.977867
1,72,79,1117.452555
2,72,82,1516.500000
3,72,83,923.157895
4,72,116,824.120000
...,...,...,...
95625,3002,2017,1394.000000
95626,3002,2021,797.652174
95627,3002,2022,2590.000000
95628,3002,2023,3319.000000


In [52]:
import pymongo
import pandas as pd

myclient=pymongo.MongoClient('mongodb://localhost:27017/')
mydb=myclient['mydatabase1'] # database
mycol=mydb['usuarios'] # colección

In [53]:
# Agregar estaciones de bicicleta a la bd

try:
  mydb.create_collection('Estaciones')
except:
  print('Ya se había creado la colección')

estaciones=mydb.get_collection('Estaciones')

Ya se había creado la colección


In [54]:
mydb.drop_collection("Estaciones")
from pymongo import  GEO2D

mydb.Estaciones.create_index([("loc", GEO2D)])

dicc_estaciones = []
for row in df_estacion.iterrows():
    idd = int(row[1]["id_estacion"])
    lat,long=row[1]["lat"], row[1]["long"]
    nombre=row[1]['nombre_estacion']

    dicc = {
        "loc":[lat,long],
                    "estacion_id": idd,
                    "estacion_nombre":nombre
        }
    

    dicc_estaciones.append(dicc)

dicc_estaciones
estaciones.insert_many(dicc_estaciones)


# Actividad 1
Un usuario se pueda dar de alta junto con información de ubicación (coordenadas), debe usar al menos un lugar (casa por ejemplo) pero con opción de agregar más (trabajo, escuela, etc.)

In [55]:
# Clase para solucionar el problema
from bson.son import SON

class Usuario(object):
  def __init__(self,diccionario, propiedades={}):
    '''
    Constructor
    El diccionario tiene que tener al menos la llave nombre, lo demás es opcional
    '''
    if 'nombre' not in diccionario.keys():
      raise ValueError('El nombre es necesario')
    self.diccionario=diccionario

  def agrega_direccion(self,nombre,coordenadas,properties):
    '''
    nombre: nombre de la ubicación (casa, trabajo, etc)
    coordenadas: la coordenada de la ubicación
    propiedades: diccionario de propiedades del objeto {'lugar':'IIMAS'}
    '''
    geoJSON={
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [coordenadas[0], coordenadas[1]]
      },
      "properties": properties
    }

    if not self.diccionario.get('ubicaciones',False):
      self.diccionario['ubicaciones']={}

    self.diccionario['ubicaciones'][nombre]=geoJSON
    return self.diccionario
  


  def agrega_BD(self, coleccion):
    '''
    Método par agregar la instancia de usuario a la base de datos
    '''
    self.coleccion=coleccion
    return coleccion.insert_one(self.diccionario).inserted_id


  def busca_estacion_cercana(self,direccion='casa',dist_max=100,num_limite=5):
    lat,long=self.diccionario['ubicaciones'][direccion]['geometry']['coordinates']
    query = {"loc": SON([("$near", [lat,long] ), ("$maxDistance", dist_max)])}
 
   
    return list(estaciones.find(query).limit(num_limite))

  def encontrar_paseo(self,ubicacion,tiempo_limite=1000,resultados_limite=5,dist_max=100,num_limite=5):
  
   # Lista de num_limite de estaciones cercanas
    lista_de_id_1=[dicc['estacion_id'] for dicc in self.busca_estacion_cercana(ubicacion,dist_max,num_limite)]

    # Listas donde guardaremos las columnas de el df
    id_intermedio=[] # El id al que se viaja en el paseo
    id_inicial=[] # El id de la estaciona donde se comienza el viaje
    duracion_total=[] # Duración total del paseo redondo

    print(lista_de_id_1)
    for id_1 in lista_de_id_1:

      # Encontramos todos los viajes que están a menos del tiempo de distancia de las estaciones cercanas
      primer_destino=viajes.find({'start_id':id_1,'duracion_prom':{'$lt':tiempo_limite}})

      # Calculamos el tiempo y los destinos que están a menos del tiempo limite tomando en cuenta el viaje redondo

      for destino in primer_destino:
        
        id_2=destino['end_id']
          
        duracion_anterior=destino['duracion_prom']

        limite_actual=tiempo_limite-duracion_anterior
        
        segundo_destino=viajes.find_one({'start_id':id_2,'end_id':id_1,'duracion_prom':{'$lt':limite_actual}})

        if segundo_destino:
          
          duracion_actual=segundo_destino['duracion_prom']

          if id_2==id_1:
            duracion_actual=duracion_anterior
            
          duracion_total.append(duracion_anterior+duracion_actual)
          id_intermedio.append(id_2)
          id_inicial.append(id_1)


    df_sort=pd.DataFrame({'id_inicial':id_inicial,'id_intermedio':id_intermedio,'duracion_redondo':duracion_total}).sort_values('duracion_redondo').iloc[:resultados_limite]

    return df_sort



Agregamos un usuario

In [57]:
u1=Usuario({'nombre':'Fernando Avitúa'})

u1.agrega_direccion('casa',[40.7192258,-73.9885016],{'lugar':'Paloma azul'})
d=u1.agrega_direccion('trabajo',[10,5],{'lugar':'IIMAS'})

u1.agrega_BD(mycol)

col=mydb.get_collection('usuarios')
col.find_one({'nombre':'Fernando Avitúa'})




ObjectId('6272c9fdddf435c41bd141d0')

In [9]:
u2 = Usuario({'nombre':'Fernando Santa Rita'})
u2.agrega_direccion('escuela',[40.689, -74],{'lugar':'Ciencias'})
u2.agrega_direccion('casa',[40.74,-73.99],{'lugar':'Contreras'})
u2.agrega_BD(mycol)

ObjectId('627337d5525714cc52e137cf')

In [10]:
u3 = Usuario({'nombre':'Guillermo Urbano'})
u3.agrega_direccion('casa', [40.689, -74], {'lugar':'Hogar'})
u3.agrega_direccion('escuela', [40.76, -73.97], {'lugar':'Hogar'})
u3.agrega_BD(mycol)

ObjectId('627337d6525714cc52e137d0')

# Actividad 2
El usuario debe poder buscar las estaciones de bicicleta que le queden más cerca a sus lugares.

In [47]:
u1.busca_estacion_cercana(direccion='casa')

[{'_id': ObjectId('6272aa72ddf435c41bd14130'),
  'loc': [40.72019576, -73.98997825],
  'estacion_id': 401},
 {'_id': ObjectId('6272aa72ddf435c41bd140dd'),
  'loc': [40.7172274, -73.98802084],
  'estacion_id': 311},
 {'_id': ObjectId('6272aa72ddf435c41bd140de'),
  'loc': [40.722055, -73.989111],
  'estacion_id': 312},
 {'_id': ObjectId('6272aa72ddf435c41bd14139'),
  'loc': [40.72066442, -73.98517977],
  'estacion_id': 410},
 {'_id': ObjectId('6272aa72ddf435c41bd14172'),
  'loc': [40.72110063, -73.9919254],
  'estacion_id': 473}]

# Actividad 3

También debe poder planear viajes, dado un tiempo que quiere viajar, el sistema debe recomendar viajes usando como salida sus estaciones mas cercanas (o estaciones específicas seleccionadas por el usuario) y los destinos que le tomen mas o menos ese tiempo.

Primero creamos la colección para la base de datos de tiempos transcurridos entre  dos estaciones, esto del Data Frame de df_viajes

In [12]:
# Agregar estaciones de bicicleta a la bd
try:
  mydb.create_collection('Tiempos')
except:
  print('Ya se había creado la colección')
tiempos = mydb.get_collection('Tiempos')

In [13]:
dicc_tiempos = []

for row in df_viajes.iterrows():
    inicio = int(row[1]['start_id'])
    fin = int(row[1]['end_id'])
    promedio = row[1]['duracion_prom']

    dicc = {
        "inicio":inicio,
        "fin": fin,
        "promedio":promedio
        }
    
    dicc_tiempos.append(dicc)

tiempos.insert_many(dicc_tiempos)

In [14]:
def viaje(usuario, partida, tiempo, ruta=None):
    
    # verifica que exista el usuario y tenga más de 2 elementos
    try:
        atributos = mydb['usuarios'].find({'nombre':usuario})[0]
    except:
        return f'El usuario no existe'
    
    # coordenadas del punto de partida
    if type(partida) == list:
        if len(partida) == 2:
            coordenadas = partida
        else:
            return f'Introduce correctamente el punto de partida'
    
    # el punto de partida es un lugar guardado por el usuario
    elif type(partida) == str:
        try:
            coordenadas = atributos['ubicaciones'][partida]['geometry']['coordinates']
        except:
            return f'Este usuario no tiene el lugar guardado'
    
    # buscamos la estación más cercana al punto de partida
    try:
        posibles = mydb.Estaciones.find({"loc": SON([("$near", coordenadas), ("$maxDistance", 0.01)])})
        closest = posibles[0]
    except:
        return f'Las estaciones están muy lejos de tí'
    
    # verificamos si el usuario prefiere pasar por ciertas estaciones
    tiempo_restante = tiempo
    coord_actual = closest['loc']
    est_nombre = closest['estacion_nombre']
    id_actual = closest['estacion_id']
    estaciones_resultado = {id_actual:[coord_actual, est_nombre]}
    
    if ruta == None:
        while tiempo_restante > 0:
            new_closest = mydb.Estaciones.find({"loc": SON([("$near", coord_actual),
                                                            ("$maxDistance", 0.5)])})

            for i in new_closest:
                if i['estacion_id'] in estaciones_resultado.keys():
                    continue
                else:
                    new_closest = i
                    break

            coord_actual = new_closest['loc']
            new_closest_id = new_closest['estacion_id']
            new_est_nombre = new_closest['estacion_nombre']
            tiempo_tomado = mydb.Tiempos.find({'inicio':id_actual, 'fin':new_closest_id})[0]['promedio']
            print(f'Toma {np.round(tiempo_tomado, 1)} segundos ir de {estaciones_resultado[id_actual][1]} a {new_est_nombre}')
            id_actual = new_closest_id
            estaciones_resultado[id_actual] = [coord_actual, new_est_nombre]
            tiempo_restante -= float(tiempo_tomado)
    
    trayectoria_x = []
    trayectoria_y = []
    for key in estaciones_resultado:
        trayectoria_x.append(estaciones_resultado[key][0][0])
        trayectoria_y.append(estaciones_resultado[key][0][1])
    fig = px.scatter(df_estacion, x='lat', y='long')        
    fig.add_trace(go.Scatter(x=trayectoria_x, y=trayectoria_y))
    fig.update_layout(template='plotly_dark', width=600)
    fig.show()
    return estaciones_resultado

In [15]:
viaje('Fernando Santa Rita', [40.689, -74], 10000)

'El usuario no existe'

In [16]:
viaje('Guillermo Urbano', 'escuela', 10000)

'El usuario no existe'

# Actividad 4 

El usuario debe poder dar la opción de que su viaje sea redondo (mismo punto de partida y salida). En este caso, solo se debe tomar en cuenta los datos que pasan por otras estaciones, a menos que el tiempo sea muy corto.

In [61]:
viajes_lista=[]
df_viajes=pd.read_csv('./Datos/datos_limpios/datos_viajes.csv').drop(columns='Unnamed: 0').rename(columns={'start station id':'start_id','end station id':'end_id','tripduration':'duracion_prom'})

mydb.drop_collection('viajes')
viajes=mydb.create_collection('viajes')

for _,row in df_viajes.iterrows():
  viajes_lista.append({'start_id':row['start_id'],'end_id':row['end_id'],'duracion_prom':row['duracion_prom']})

viajes.insert_many(viajes_lista)

Encontramos la lista de paseos redondos que salen de las cinco estaciones iniciales más cercanas al punto de partida. 

In [63]:
df1=u1.encontrar_paseo('casa')

[401, 311, 312, 410, 473]


In [131]:
df1

,id_inicial,id_intermedio,duracion_redondo
33,311,502.0,554.067117
30,311,356.0,569.068049
59,410,317.0,631.294992
53,410,150.0,639.230876
28,311,340.0,646.567980


## Gráfica de viajes redondos

In [102]:
x_inicial=[]
y_inicial=[]
for id1 in df1.id_inicial.unique():
  x_inicial.append(df_estacion[df_estacion.id_estacion==id1].lat.iloc[0])
  y_inicial.append(df_estacion[df_estacion.id_estacion==id1].long.iloc[0])
y_inicial

x_intermedio=[]
y_intermedio=[]
for id1 in df1.id_intermedio.unique():
  x_intermedio.append(df_estacion[df_estacion.id_estacion==id1].lat.iloc[0])
  y_intermedio.append(df_estacion[df_estacion.id_estacion==id1].long.iloc[0])
y_intermedio

[-73.981346, -73.98261206, -73.98185424, -73.98085795, -73.98776323]

In [130]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
x_partida,y_partida=u1.diccionario['ubicaciones']['casa']['geometry']['coordinates']
plt.scatter(x=df_estacion.lat, y=df_estacion.long,s=20,label='Estaciones')        
plt.scatter(x=x_inicial, y=y_inicial,s=150,label='Estaciones iniciales',marker='*')        
plt.scatter(x=x_intermedio, y=y_intermedio,s=150,label='Estaciones intermedias',marker='*')        
plt.scatter(x=x_partida, y=y_partida,s=200,label='Punto partida')   
plt.title('Viajes Redondos con tiempo menor a 1000',fontsize=20)     
# fig.add_trace(go.Scatter(x=trayectoria_x, y=trayectoria_y))
plt.legend()

# Actividad 5 (Extra)

Si el usuario busca una ruta específica que el sistema no tenga, la debe intentar armar de forma indirecta.

# Actividad 6 (Extra, después)

Los resultados deben tomar en cuenta la hora a la que se hace la consulta y la hora de los viajes registrados para dar sus resultados con mayor confianza.

## Queries ejemplo

In [77]:
import pprint
for doc in mydb.Estaciones.find({"loc": {"$near": [40.767272,-73.993929]}}).limit(3):
  pprint.pprint(doc)

{'_id': ObjectId('6272aa72ddf435c41bd1407a'),
 'estacion_id': 72,
 'loc': [40.76727216, -73.99392888]}
{'_id': ObjectId('6272aa72ddf435c41bd14179'),
 'estacion_id': 480,
 'loc': [40.76669671, -73.99061728]}
{'_id': ObjectId('6272aa72ddf435c41bd14187'),
 'estacion_id': 495,
 'loc': [40.76269882, -73.99301222]}


In [84]:
from bson.son import SON
query = {"loc": SON([("$near", [3, 6]), ("$maxDistance", 100)])}
for doc in mydb.Estaciones.find(query).limit(3):
  pprint.pprint(doc)

{'_id': ObjectId('6272aa72ddf435c41bd14150'),
 'estacion_id': 437,
 'loc': [40.6809833854, -73.9500479759]}
{'_id': ObjectId('6272aa72ddf435c41bd1414f'),
 'estacion_id': 436,
 'loc': [40.68216564, -73.95399026]}
{'_id': ObjectId('6272aa72ddf435c41bd1412b'),
 'estacion_id': 396,
 'loc': [40.680342423, -73.9557689392]}


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9aa4e14a-adbe-4d9d-8b09-68340723a69a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>